# Example KubeHound queries 

This file contains examples queries for kubehound. This file should be mostly read only. No modification will be ported to the original file, it will be kept only for the duration of the docker container.

Note: You may need to adjust the value on the cell below and execute (ctrl + enter) before running any other cells.

In [ ]:
%%graph_notebook_config
{
  "host": "kubehound-testing-graphdb",
  "port": 8182,
  "ssl": false,
  "gremlin": {
    "traversal_source": "g",
    "username": "",
    "password": "",
    "message_serializer": "graphsonv3"
  }
}

# All paths between a volume and identity

In [ ]:
%%gremlin
g.V().hasLabel("Volume").repeat(out().simplePath()).until(hasLabel("Identity")).path()

# All container escapes

In [ ]:
%%gremlin
g.V().hasLabel("Container").repeat(out().simplePath()).until(hasLabel("Node")).path()